In [1]:
import torch
from gpytorch.means import ConstantMean
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.variational import WhitenedVariationalStrategy, CholeskyVariationalDistribution
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import AbstractVariationalGP
from gpytorch.mlls import VariationalELBO, AddedLossTerm
from gpytorch.likelihoods import GaussianLikelihood

In [2]:
from deep_gp import DeepGP, DeepGaussianLikelihood

In [3]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor
import numpy as np

if not os.path.isfile('elevators.mat'):
    print('Downloading \'elevators\' UCI dataset...')
    urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jhWL3YUHvXIaftia4qeAyDwVxo6j1alk', 'elevators.mat')
    
data = torch.Tensor(loadmat('elevators.mat')['data'])
X = data[:, :-1]
y = data[:, -1]

N = data.shape[0]
np.random.seed(0)
data = data[np.random.permutation(np.arange(N)),:]

train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

mean = train_x.mean(dim=-2, keepdim=True)
std = train_x.std(dim=-2, keepdim=True) + 1e-6
train_x = (train_x - mean) / std
test_x = (test_x - mean) / std

mean,std = train_y.mean(),train_y.std()
train_y = (train_y - mean) / std
test_y = (test_y - mean) / std

In [4]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

In [9]:
num_samples = 1

model = DeepGP(input_dims=18, hidden_dims=10, output_dims=1, num_inducing=200, num_samples=num_samples).cuda()
likelihood = DeepGaussianLikelihood(num_samples=num_samples).cuda()
mll = VariationalELBO(likelihood, model, train_x.size(-2))

In [ ]:
num_epochs = 40

optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.01)

for i in range(num_epochs):
    # Within each iteration, we will go over each minibatch of data
    for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(x_batch)
        print(output.mean.shape)
        y_batch = y_batch.unsqueeze(0).unsqueeze(0).expand(model.output_dims, model.num_samples, y_batch.size(-1))
        loss = -mll(output, y_batch).sum()
        
        print('Epoch %d [%d/%d] - Loss: %.3f' % (i + 1, minibatch_i, len(train_loader), loss.item()))

        loss.backward()
        optimizer.step()

torch.Size([1, 1024])
Epoch 1 [0/13] - Loss: 1.998
torch.Size([1, 1024])
Epoch 1 [1/13] - Loss: 2.012
torch.Size([1, 1024])
Epoch 1 [2/13] - Loss: 1.999
torch.Size([1, 1024])
Epoch 1 [3/13] - Loss: 1.875
torch.Size([1, 1024])
Epoch 1 [4/13] - Loss: 1.926
torch.Size([1, 1024])
Epoch 1 [5/13] - Loss: 1.841
torch.Size([1, 1024])
Epoch 1 [6/13] - Loss: 1.871
torch.Size([1, 1024])
Epoch 1 [7/13] - Loss: 1.810


In [26]:
preds = likelihood(model(test_x))

In [31]:
torch.sqrt(torch.mean(torch.pow(preds.mean.reshape(model.num_samples, -1).mean(0) - test_y, 2)))

tensor(0.9957, device='cuda:0', grad_fn=<SqrtBackward>)

In [27]:
test_y

tensor([-0.3685, -0.1655,  1.0216,  ...,  2.6740, -0.8084, -0.1655],
       device='cuda:0')

In [29]:
preds.mean.reshape(model.num_samples, -1)

torch.Size([3, 3320])

In [30]:
test_y.shape

torch.Size([3320])